# **Respondent Profile Simulator for MMBN-CAS Questionnaire**

---

### 1  Sample per replica

| Maturity Profile      | Personas (executive)                    | Synthetic “Respondents” per replica | Justification                                                                         |
| --------------------- | --------------------------------------- | ----------------------------------- | ------------------------------------------------------------------------------------- |
| Beginner (µ = –0.5)   | Sustainability, Technology, Operations  | 60 (20 + 20 + 20)                   | GRM studies indicate stability of parameters a and b with ≥ 150 cases per dimension¹   |
| Intermediate (µ = 0)  | Sustainability, Technology, Operations  | 60                                  | Same logic                                                                           |
| Advanced (µ = 0.8)    | Sustainability, Technology, Operations  | 60                                  | Symmetric coverage of the distribution of θ                                          |

**Total per replica** = 180 responses × 22 items ≈ 4,000 model calls.


### 2  Number of Monte Carlo replicas

* **20 independent replicas** (different seeds) generate a total of 3,600 “respondents,” producing about 80,000 item×person observations.
* Literature in psychometric simulation shows that 20–30 replicas allow for estimating bias and RMSE with a standard error < 0.01 for samples of this size¹.


### 3  Maintained metrics

* |bias| < 0.05 and RMSE < 0.10 for GRM parameters.
* CFI/TLI > 0.95; RMSEA < 0.06.
* α and ω ≥ 0.80 per dimension.
* Total information > 15 in |θ| ≤ 1.5.
* Stress-test: same rule, but applied to the 20 concatenated replicas.


### 4  Impact on duration and cost

* Considering an average time of 1.5 s per call, each replica consumes ~100 min, but can be parallelized in batches of 1,000 calls.
* With 20 replicas distributed across ten cores, the experiment can be completed in ~3 h.
* The total token consumption is reduced by more than 95% compared to the initial plan.

---

¹ **Key Reference**
MORRIS, T. P.; WHITE, I. R.; CROWTHER, M. J. *Using Simulation Studies to Evaluate Statistical Methods*. arXiv:1712.03198, 2018.


## **Setup and Personas**

---

* ECCLES, R. G.; TAYLOR, A. The evolving role of chief sustainability officers. Harvard Business Review, Boston, v. 101, n. 4, p. 1-9, jul./ago. 2023. 
hbr.org
* CHANG, A.; EL-RAYES, N.; SHI, J. J. Blockchain technology for supply chain management: a comprehensive review. FinTech, Basel, v. 1, n. 2, p. 191-205, 2022. DOI: 10.3390/fintech1020015. 
researchgate.net
* CHARLEBOIS, S. et al. Digital traceability in agri-food supply chains: a comparative analysis of OECD member countries. Foods, Basel, v. 13, n. 7, art. 1075, 2024. DOI: 10.3390/foods13071075. 
mdpi.com
* JOVANOVIC, M. et al. Managing a blockchain-based platform ecosystem for industry-wide adoption: the case of TradeLens. Technological Forecasting and Social Change, Amsterdam, v. 186, art. 121981, 2022. DOI: 10.1016/j.techfore.2022.121981. 
arxiv.org

In [12]:
# ======= Initial setup (imports + .env) =======
"""
Imports, environment loading, and OpenAI key configuration
Comments in English, no accents or cedillas
"""

import os
import sys
import time
import json
import random
from typing import Dict, List, Any, Optional   # Optional added

import numpy as np
import pandas as pd
from dotenv import load_dotenv
from tqdm.auto import tqdm                    # single tqdm import
import openai
from tenacity import retry, stop_after_attempt, wait_exponential

# Path to your .env file
DOTENV_PATH = "/mnt/4d4f90e5-f220-481e-8701-f0a546491c35/arquivos/projetos/.envpaulo"

# Load environment variables
load_dotenv(dotenv_path=DOTENV_PATH)

# Configure OpenAI API key
openai.api_key = os.getenv("OPENAI_API_KEY")
if not openai.api_key:
    sys.exit("ERROR: OPENAI_API_KEY not found in environment variables.")


In [13]:
"""
MMAB-NCAS Meta-validation Notebook
Full standalone script – ready for a single Jupyter cell
Comments in English, no accents or cedillas
"""

# ============================================================ #
# 1. Imports and global configuration
# ============================================================ #
import random
from typing import Dict, List, Tuple, Any
import numpy as np

# ---------- Global experiment parameters -------------------- #
N_REPLICAS          = 20          # Monte Carlo iterations
RESP_PER_PROFILE    = 60          # synthetic respondents per profile per replica
ITEMS_PER_RESP      = 22          # questionnaire length
SEED_BASE           = 42          # reproducible base seed
MODEL_NAME          = "gpt-4.1-mini"

# ---------- Seed PRNGs -------------------------------------- #
random.seed(SEED_BASE)
np.random.seed(SEED_BASE)

# ---------- Theta distributions by profile ------------------ #
PROFILE_CONFIG: Dict[str, Dict[str, float]] = {
    "novice":       {"mu": -0.5, "sigma": 0.7},
    "intermediate": {"mu":  0.0, "sigma": 0.7},
    "advanced":     {"mu":  0.8, "sigma": 0.7}
}

# ============================================================ #
# 2. Personas metadata
# ============================================================ #
PERSONAS: Dict[str, Dict[str, Any]] = {
    "strategy_sustainability": {
        "role": "Chief Sustainability Officer",
        "sector": "Agro-food supply chain",
        "core_objectives": [
            "Reduce GHG emissions",
            "Ensure ESG compliance",
            "Enhance corporate reputation"
        ]
    },
    "strategy_technology": {
        "role": "Chief Technology Officer",
        "sector": "Agro-food supply chain",
        "core_objectives": [
            "Modernize digital architecture",
            "Integrate blockchain with legacy systems",
            "Strengthen data governance"
        ]
    },
    "operations": {
        "role": "Director of Operations",
        "sector": "Agro-food supply chain",
        "core_objectives": [
            "Optimize logistics cost",
            "Guarantee end-to-end traceability",
            "Maintain quality certifications"
        ]
    }
}

# ============================================================ #
# 3. Canonical questionnaire (exactly 22 items)
#    Tuple format: (maturity level, item stem)
# ============================================================ #
FINAL_ITEMS: List[Tuple[int, str]] = [
    # Level 1 – Viability (4)
    (1, "Pilot projects: Access to product certification"),
    (1, "Blockchain transaction and record technology: Tamper-resistant records"),
    (1, "Applications in agricultural production: Agility and productivity increase"),
    (1, "Credit and financing in the agricultural market: Quality and safety credit"),
    # Level 2 – Alignment (4)
    (2, "Information gathering about technology usage: Information sharing"),
    (2, "Assessment of participation in the process: Producer sales"),
    (2, "Search for suppliers specialized in the technology: Start-ups specialized in blockchain deployment"),
    (2, "Tracking of batches destined for export: Types of certifications"),
    # Level 3 – Standardized (4)
    (3, "Credit asset contracts: Consensus mechanisms"),
    (3, "Product exchanges via blockchain across the supply chain: Information management"),
    (3, "Integration of digital platforms: Innovative technologies"),
    (3, "Use of tokens to facilitate trading and financial advances: Asset tokenization in transactions"),
    # Level 4 – Trustworthy (5)
    (4, "Carbon credit market: Carbon emission registry"),
    (4, "Secure data sharing in integrated supply chains: Sustainable trust"),
    (4, "Carbon footprint and genetic information: Genetic identification of food"),
    (4, "Agtech platforms for digital production and commercialization: Productivity increase"),
    (4, "Traceability in inter-enterprise ecosystems: Transparency"),
    # Level 5 – Continuous improvement (5)
    (5, "Traceability fully embedded in day-to-day operations: Transaction ledger"),
    (5, "Sociotechnical solutions"),
    (5, "Sustainable ownership: Environmental, social and economic indicators"),
    (5, "Multisector export chains with digital certification: Distributed data registry"),
    (5, "Production tokenization: Use of Non-Fungible Tokens (NFTs)")
]

TOTAL_ITEMS: int = len(FINAL_ITEMS)              # must be 22
FULL_Q_TEXT: str = "\n".join(
    f"{idx+1}. {stem}" for idx, (_, stem) in enumerate(FINAL_ITEMS)
)

# ============================================================ #
# 4. Prompt builder (full questionnaire embedded)
# ============================================================ #
def build_prompt_full_q(
    persona_id: str,
    theta_value: float,
    questionnaire_id: str = "MMAB-NCAS-22",
    scale_labels: Optional[List[str]] = None  # Change made here
) -> str:
    """
    Build a GPT-4-1-mini prompt that embeds the 22-item questionnaire and
    asks for a single response to an item (1-based).
    """
    default_labels = [
        "1 = very low",
        "2 = low",
        "3 = moderate",
        "4 = high",
        "5 = very high"
    ]
    labels = scale_labels if scale_labels else default_labels

    persona = PERSONAS[persona_id]
    role, sector = persona["role"], persona["sector"]
    mandate = persona["core_objectives"][0].lower()
    item_stem = FINAL_ITEMS[0][1]  # Default to the first item for the prompt

    prompt = f"""
        Questionnaire: {questionnaire_id}

        Below is the full list of {TOTAL_ITEMS} statements that compose this instrument.
        Read them for context. Then focus in all items from 1 to 22 and choose a single
        number from the response scale that best represents your organisation.

        --- START OF QUESTIONNAIRE ---

        {FULL_Q_TEXT}
        
        --- END OF QUESTIONNAIRE ---

        You are a {role} working in the {sector}.
        Your overall blockchain maturity level is theta = {theta_value:.2f} (standard normal).

        Response scale:
        {', '.join(labels)}

        Return only the chosen number (1-5). Do not add explanations.
        Respond with a JSON in the format: {{
        "1": "response", "2": "response", "3": "response", "4": "response", "5": "response",
        "6": "response", "7": "response", "8": "response", "9": "response", "10": "response",
        "11": "response", "12": "response", "13": "response", "14": "response", "15": "response",
        "16": "response", "17": "response", "18": "response", "19": "response", "20": "response",
        "21": "response", "22": "response"
        }}
    """.strip()

    return prompt


In [14]:
# Quick sanity test
# choose persona, maturity profile, and random item
persona_id  = "strategy_technology"
profile_key = "intermediate"
mu, sigma   = PROFILE_CONFIG[profile_key]["mu"], PROFILE_CONFIG[profile_key]["sigma"]
theta_val   = np.random.normal(loc=mu, scale=sigma)
prompt_test = build_prompt_full_q(
    persona_id   = persona_id,
    theta_value  = theta_val
)

print(prompt_test)
    

Questionnaire: MMAB-NCAS-22

        Below is the full list of 22 statements that compose this instrument.
        Read them for context. Then focus in all items from 1 to 22 and choose a single
        number from the response scale that best represents your organisation.

        --- START OF QUESTIONNAIRE ---

        1. Pilot projects: Access to product certification
2. Blockchain transaction and record technology: Tamper-resistant records
3. Applications in agricultural production: Agility and productivity increase
4. Credit and financing in the agricultural market: Quality and safety credit
5. Information gathering about technology usage: Information sharing
6. Assessment of participation in the process: Producer sales
7. Search for suppliers specialized in the technology: Start-ups specialized in blockchain deployment
8. Tracking of batches destined for export: Types of certifications
9. Credit asset contracts: Consensus mechanisms
10. Product exchanges via blockchain across the

## **Drives the full Monte Carlo simulation**

It organises outputs in a pandas DataFrame with columns:
* replica, profile, respondent_id, persona, theta, item_number, response.
* 20 replicas x 3 maturity profiles x 60 respondents.
* Total prompts: 3600.


In [15]:
# ======= Chunk 6-params: prepare agenda, no LLM calls =======
"""
Creates a DataFrame with the parameters needed for build_prompt.
Columns: replica | profile | respondent_id | persona | theta | output (None).
No API calls are made here.
"""

import uuid, pandas as pd

agenda_records = []

for replica in range(1, N_REPLICAS + 1):
    for profile_key, cfg in PROFILE_CONFIG.items():
        # Sample 60 theta values for this profile
        thetas = np.random.normal(
            loc=cfg["mu"],
            scale=cfg["sigma"],
            size=RESP_PER_PROFILE
        )

        # Round-robin assign personas
        persona_ids = list(PERSONAS.keys())
        assigned_personas = [
            persona_ids[i % len(persona_ids)] for i in range(RESP_PER_PROFILE)
        ]

        # Build agenda rows
        for theta_val, persona_id in zip(thetas, assigned_personas):
            persona_info = PERSONAS[persona_id]  # Get full persona information
            agenda_records.append(
                {
                    "replica":        replica,
                    "profile":        profile_key,
                    "respondent_id":  str(uuid.uuid4()),
                    "persona":        f"persona: {persona_id} {persona_info['role']} {persona_info['sector']} {persona_info['core_objectives']}",
                    "theta":          float(theta_val),
                    "output":         None          # will store JSON later
                }
            )

# Convert to DataFrame
df_agenda = pd.DataFrame.from_records(agenda_records)

# Quick sanity check
print(df_agenda.head())
print(f"\nTotal rows: {len(df_agenda)}  (expected {N_REPLICAS*3*RESP_PER_PROFILE})")


   replica profile                         respondent_id  \
0        1  novice  1ce5c5cc-c230-455b-823b-48a26b52f83f   
1        1  novice  15ba9701-5eba-47e4-918b-434ab1f955bd   
2        1  novice  2bb6be40-aa4e-4e55-9c40-7faae6c96c23   
3        1  novice  d4aef519-2a51-4e5f-84b6-000d2cfcbc53   
4        1  novice  bbefe492-0d6b-4e37-a041-f3a66fd8da18   

                                             persona     theta output  
0  persona: strategy_sustainability Chief Sustain... -0.596785   None  
1  persona: strategy_technology Chief Technology ... -0.046618   None  
2  persona: operations Director of Operations Agr...  0.566121   None  
3  persona: strategy_sustainability Chief Sustain... -0.663907   None  
4  persona: strategy_technology Chief Technology ... -0.663896   None  

Total rows: 3600  (expected 3600)


In [18]:
"""
After execution, df_agenda will contain 3 600 rows with
the JSON answers (dict) stored in the 'output' column.
"""

from tqdm import tqdm  # Import tqdm for progress bar

MODEL_NAME_CALL = MODEL_NAME
MAX_TOKENS      = 150

# Robust call with retries
@retry(stop=stop_after_attempt(3), wait=wait_exponential(multiplier=2, min=2, max=20))
def call_llm(prompt: str) -> dict:
    messages = [{"role": "system", "content": prompt}]
    response = openai.ChatCompletion.create(
        model       = MODEL_NAME_CALL,
        messages    = messages,
        max_tokens  = MAX_TOKENS
    )
    result = response["choices"][0]["message"]["content"]
    return result

# Process only the first two entries of df_agenda with a progress bar
for index in tqdm(range(len(df_agenda)), desc="Processing rows", unit="row"):  # Iterate over all rows with progress bar
    row = df_agenda.iloc[index]
    if row["output"] is None:  # Check if it hasn't been processed
        persona_info = row["persona"].split(" ")  # Split the persona string
        persona_id = persona_info[1]  # Extract persona_id
        if persona_id not in PERSONAS:  # Check if persona_id exists in PERSONAS
            raise KeyError(f"Persona ID '{persona_id}' not found in PERSONAS.")
        prompt_txt = build_prompt_full_q(
            persona_id  = persona_id,
            theta_value = row["theta"]
        )
        try:
            answers_dict = call_llm(prompt_txt)
            df_agenda.at[index, "output"] = answers_dict  # Update the row
        except Exception as e:
            print(f"Failed row {index}: {e}")
            df_agenda.at[index, "output"] = None   # mark as failed; retry later or inspect

# Persist results
df_agenda.to_json("outputs/mmbncas_llm_raw.jsonl", orient="records", lines=True)
print("\nFinished. Saved to mmbncas_llm_raw.jsonl")


Processing rows: 100%|██████████| 3600/3600 [2:18:09<00:00,  2.30s/row]  


Finished. Saved to mmbncas_llm_raw.jsonl


In [17]:
#%pip install -r requirements.txt